In [1]:
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.sparse import save_npz

import json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import nltk

import numpy as np

In [2]:
data_dir = '2021-07-19'

In [3]:
data = pd.read_csv(f'scibert编码文章/needed_paper_metadata_{data_dir}.csv')

In [4]:
data.fillna(' ',inplace=True)

In [5]:
# 包含哪些词
words = []

In [6]:
for idx, row in tqdm(data.iterrows()):
    t_tokens = nltk.word_tokenize(row['title'].lower())
    a_tokens = nltk.word_tokenize(row['abstract'].lower())
    data.at[idx, 'title'] = t_tokens
    data.at[idx, 'abstract'] = a_tokens
    words.extend(t_tokens)
    words.extend(a_tokens)

84391it [01:55, 727.97it/s]


In [7]:
word_set = set(words)

In [8]:
len(word_set)

258035

In [9]:
# 读 embedding

In [10]:
word_embs = []
with open('glove.42B.300d.txt','r',encoding='utf-8') as fr:
    for line in tqdm(fr):
        line = line.strip().split(' ')
        word = line[0]
        if word in word_set:
#             embed = map(lambda x:float(x),line[1:])
            word_embs.append(line)

1917494it [00:29, 64542.88it/s]


In [11]:
word_embs = pd.DataFrame(word_embs).set_index(0).astype(float)

In [12]:
final_word_set = set(word_embs.index)

In [13]:
title_embs = []
abstract_embs = []

for idx, row in tqdm(data.iterrows()):
    title_emb = word_embs.loc[list(filter(lambda x:x in final_word_set,row['title']))].mean(0).values
    title_embs.append(title_emb)
    
    abstract_emb = word_embs.loc[list(filter(lambda x:x in final_word_set,row['abstract']))].mean(0).values
    abstract_embs.append(abstract_emb)

84391it [05:11, 271.05it/s]


In [14]:
title_embs_array = np.vstack(title_embs)
title_embs_df = pd.DataFrame(data=title_embs_array,columns=['emb'+str(i) for i in range(300)],index=data['cord_uid']).fillna(0.)

In [15]:
abstract_embs_array = np.vstack(abstract_embs)
abstract_embs_df = pd.DataFrame(data=abstract_embs_array,columns=['emb'+str(i) for i in range(300)],index=data['cord_uid']).fillna(0.)

In [16]:
title_embs_df.to_csv(f'title_embs_df_glove_{data_dir}.csv')
abstract_embs_df.to_csv(f'abstract_embs_df_glove_{data_dir}.csv')